In [1]:
import sys
sys.path.append('./models')

In [ ]:
import torch
import onnx
from models import Cnn14

In [7]:
print(torch.__version__)
print(onnx.__version__)

2.6.0+cu126
1.17.0


In [8]:
torch.cuda.is_available()

True

In [9]:
CHECKPOINT = "./checkpoint/Cnn14_16k_mAP=0.438.pth" # 모델의 사전학습된 가중치
device = 'cuda' if torch.cuda.is_available() else 'cpu' # cuda 사용 가능여부 확인
# print(device)

model = Cnn14(
    sample_rate=16000, # 샘플링 레이트 16k
    window_size=512, # 윈도우 사이즈
    hop_size=160, # 홉 사이즈
    mel_bins=64, # mel 주파수 채널 수
    fmin=50, # mel 주파수 최소치
    fmax=8000, # mel 주파수 최대치
    classes_num=527 # 분류할 클래스 숫자
)
checkpoint = torch.load(CHECKPOINT, map_location=device, weights_only=False) # 사전학습된 가중치 로드
model.load_state_dict(checkpoint['model'])
model.to(device)
model.eval() # 모델을 추론모드로 변경
# print(model)

Cnn14(
  (spectrogram_extractor): Spectrogram(
    (stft): STFT(
      (conv_real): Conv1d(1, 257, kernel_size=(512,), stride=(160,), bias=False)
      (conv_imag): Conv1d(1, 257, kernel_size=(512,), stride=(160,), bias=False)
    )
  )
  (logmel_extractor): LogmelFilterBank()
  (spec_augmenter): SpecAugmentation(
    (time_dropper): DropStripes()
    (freq_dropper): DropStripes()
  )
  (bn0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv_block1): ConvBlock(
    (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv_block2): ConvBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (con

In [10]:
'''
STFT 입력층의 패딩의 형태를 상수의 형대로 정하는 코드
모델의 기본값은 "reflect" 이나
onnx 와의 호환성을 위해 "constant" 변경
'''
model.spectrogram_extractor.stft.pad_mode = "constant"

In [ ]:
dummy_input = torch.randn(1, 16000, device=device) # 모델의 입력 텐서의 예시 현재 모델은 (배치사이즈, 길이)로 구성

In [ ]:
# onnx 모델 변환
torch.onnx.export(
    model,
    dummy_input,
    'panns_cnn14.onnx', # 저장 경로
    input_names=['input'], # 입력 노드 이름
    output_names=['output'], # 출력 노드 이름
    dynamic_axes={'input': {0: 'batch_size'}, 'output': {0: 'batch_size'}}, # 입력과 출력의 0차원(배치사이즈)의 동적입력을 위한 설정
    opset_version=11 # onnx의 연산자 버전
)

c:\Users\Threadnull\anaconda3\envs\pytorch_2.6.0\Lib\site-packages\torch\onnx\_internal\jit_utils.py:308: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\jit\passes\onnx\constant_fold.cpp:180.)
  _C._jit_pass_onnx_node_shape_type_inference(node, params_dict, opset_version)
c:\Users\Threadnull\anaconda3\envs\pytorch_2.6.0\Lib\site-packages\torch\onnx\utils.py:657: UserWarning: Constant folding - Only steps=1 can be constant folded for opset >= 10 onnx::Slice op. Constant folding not applied. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\jit\passes\onnx\constant_fold.cpp:180.)
  _C._jit_pass_onnx_graph_shape_type_inference(
c:\Users\Threadnull\anaconda3\envs\pytorch_2.6.0\Lib\site-packages\torch\onnx\utils.py:1127: UserWarning: Constant folding - Only steps=1 can be constant folded for o

In [1]:
# 모델 시각화
import netron
netron.start ("onnx/panns_cnn14.onnx")

Serving 'onnx/panns_cnn14.onnx' at http://localhost:8080


('localhost', 8080)